In [1]:
import numpy as np
import pandas as pd
import requests
from lxml import html, etree

### Selenium Step

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep

In [3]:
urls = []

chrome_options = Options()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--headless')    

In [4]:
driver = webdriver.Chrome(options = chrome_options)
#driver.maximize_window()
driver.get("https://www.plazavea.com.pe/tecnologia/telefonia/celulares-y-smartphones")

num_pagina = 1

while(num_pagina<=30):
    try:
        if num_pagina == 1:
            boton_cookies = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[@class='cookies-consent-banner__button']")) )
            boton_cookies.click()

            '''combo_marca = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "//span[@class='control__label__text' and contains(text(), 'XIAOMI')]")) )
            combo_marca.click()
            '''
        
        links_celulares = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='ShowcaseGrid']/*/*/*/div[@class='Showcase__productImage']/a")) )

        for link_cel in links_celulares:
            urls.append(link_cel.get_attribute('href') if link_cel.get_attribute('href') else 'No encontrado')

        btn_next = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='pagination__item page-control next ']")) )  
        
        url_actual = driver.current_url
        btn_next.click()

        #sleep(random.uniform(5, 7))
        url_actual = driver.current_url
        WebDriverWait(driver, 15).until(lambda driver: driver.current_url != url_actual)

    except Exception as e:
        print(str(e))

    num_pagina+=1

driver.quit()

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 



In [5]:
len(list(set(urls)))

602

In [6]:
list(set(urls))

['https://www.plazavea.com.pe/smartphone-xiaomi-redmi-note-13-pro-5g-6-67--12gb-512gb-200mp8mp2mp-blanco/p',
 'https://www.plazavea.com.pe/celular-honor-200-lite-8gb-ram-256gb-6-7--starry-blue-earbuds-x5-101095351/p',
 'https://www.plazavea.com.pe/infinix-gt-20-pro-5g-12gb-ram-256gb-rom-silver-101101527/p',
 'https://www.plazavea.com.pe/smartphone-motorola-g34-6-5--8gb-256gb-50mp2mp-negro-meteorito/p',
 'https://www.plazavea.com.pe/iphone-13-reacondicionado-256gb-medianoche-101071531/p',
 'https://www.plazavea.com.pe/iphone-14-reacondicionado-128gb-rojo-101071603/p',
 'https://www.plazavea.com.pe/celular-zte-blade-a54-4gb-ram-128gb-gris-101085398/p',
 'https://www.plazavea.com.pe/celular-motorola-moto-g54-8gb-256gb-negro-101056215/p',
 'https://www.plazavea.com.pe/celular-redmi-note-13-6-67--8gb-256gb-midnight-black-100971338/p',
 'https://www.plazavea.com.pe/iphone-15-pro-max-512gb-esim-black-titanium-101053569/p',
 'https://www.plazavea.com.pe/iphone-12-reacondicionado-64gb-verde-101

In [7]:
start_urls = list(set(urls))

### Scrapy step

In [8]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from scrapy.crawler import CrawlerProcess

import pandas as pd

import urllib.parse

import re

In [9]:
class celular(Item):
    nombre = Field()
    marca = Field()
    sku = Field()
    precio = Field()
    precio_online = Field()
    bateria = Field()
    ram = Field()
    capacidad = Field()
    sistema_operativo = Field()

In [10]:
class CatalogoCel(CrawlSpider):
    name = 'celulares'
    custom_settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'FEED_EXPORT_ENCODING': 'utf-8', # Para evitar problemas con codificacion de simbolos
    'FEED_EXPORT_FIELDS': ['nombre','marca','sku','precio','precio_online','bateria','ram','capacidad','sistema_operativo'],
    'DOWNLOAD_DELAY': 1.5,
    'CONCURRENT_REQUESTS': 2,
    'FEED_EXPORT_DELIMITER': '|'
    }

    # Url semillas

    start_urls = start_urls
    
    def parse_start_url(self, response):
        sel = Selector(response)
        item = ItemLoader(celular(),sel)

        #Obtener los elementos
        nombre = sel.xpath("//h1[@class='ProductCard__name']/div[contains(@class,'productName')]/text()").get()
        item.add_value('nombre',nombre)

        marca = sel.xpath("//div[contains(@class,'brandName')]/a/text()").get()
        item.add_value('marca',marca)

        sku = sel.xpath("//div[contains(@class,'productReference')]/text()").get()
        item.add_value('sku',sku)

        precio = sel.xpath("//span[@class='ProductCard__price__integer']/text()").get()
        item.add_value('precio',precio)

        precio_online = sel.xpath("//input[@id='___rc-p-dv-id']/@value").get()
        item.add_value('precio_online',precio_online)

        bateria = sel.xpath("//th[@class='name-field Bateria']/../td/text()").get()
        item.add_value('bateria',bateria)

        ram = sel.xpath("//th[@class='name-field Memoria-RAM']/../td/text()").get()
        item.add_value('ram',ram)

        capacidad = sel.xpath("//th[@class='name-field Memoria-interna']/../td/text()").get()
        item.add_value('capacidad', capacidad)

        sistema_operativo = sel.xpath("//th[@class='name-field Compatibilidad']/../td/text()").get()
        item.add_value('sistema_operativo', sistema_operativo)


        yield item.load_item()

In [11]:
# Configura el proceso especificando FEEDS para evitar guardar en CSV
process = CrawlerProcess( settings= {'FEEDS': {'data_plazavea.csv': {'format':'csv'}}} )

# Inicia el crawling
process.crawl(CatalogoCel)
process.start()

2024-09-11 23:33:41 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-09-11 23:33:41 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Windows-10-10.0.18362-SP0
2024-09-11 23:33:41 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-11 23:33:41 [py.warnings] WARNING: d:\Carpetas Profesionales\ProyectosVSCode\PrimerProyecto\env\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a futur